<a href="https://colab.research.google.com/github/ulusberrin/INM706_CW/blob/yasinunsal/sentiment_analysis_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab\ Notebooks
!ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
 05180000029.ipynb
 checkpoint.pth
'Copy of sentiment_analysis_lstm_dropout_05_hidden_size_16_batch_size_256.ipynb'
 FasterRCNN_112_112_faster_weights.ipynb
 FasterRCNN_112_112.ipynb
 FasterRCNN_112_112_mobilenet_v3_large_320_fpn.ipynb
 FasterRCNN_224_224.ipynb
 FasterRCNN_448_448.ipynb
 faster_rcnn_plots_and_summaries
 fruit-images-for-object-detection
'IMDB Dataset.csv'
 INM702-Task2.ipynb
'INM706_Lab7_Bert_classification (1).ipynb'
 INM706_Lab7_Bert_classification.ipynb
 sentiment_analysis_gru.ipynb
 sentiment_analysis_inference.ipynb
 sentiment_analysis_lstm_dropout_02.ipynb
 sentiment_analysis_lstm_dropout_05_hidden_size_128.ipynb
 sentiment_analysis_lstm_dropout_05_hidden_size_16_batch_size_256.ipynb
 sentiment_analysis_lstm_dropout_05_hidden_size_16_batch_size_64.ipynb
 sentiment_analysis_lstm_dropout_05_hidden_si

In [18]:
import pandas as pd
import numpy as np
import re
import string

import torch
from torch import nn
import tensorflow as tf

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stopwords = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [20]:
def remove_html(review):
  review = review.replace("<br /><br />", "")
  return review

def remove_punctuations(review):
  for punctuation in string.punctuation:
    review = review.replace(punctuation, "")
    return review

def remove_digits(review):
  review = "".join(y for y in review if not y.isdigit())
  return review

def remove_nonascii(review):
  review = re.sub(r'[^\x00-\x7f]', r'', review)
  return review

def make_lower(review):
  review = review.lower()
  return review

def remove_stop_words(review):
  review = ' '.join([word for word in review.split() if word not in (stopwords)])
  return review


def clean_review(review):
  review = remove_html(review)
  review = remove_punctuations(review)
  review = remove_digits(review)
  review = remove_nonascii(review)
  review = make_lower(review)
  review = remove_stop_words(review)
  return review


In [21]:
num_words = 10000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words)

maxlen = 256
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [22]:
class LSTMModel(nn.Module):
  def __init__(
    self,
    vocab_size,
    output_size,
    hidden_size,
    embedding_size,
    n_layers,
    dropout
    ):

    super(LSTMModel, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_size)
    #self.dropout = nn.Dropout(dropout)
    self.dropout1d = nn.Dropout1d(dropout)
    self.lstm = nn.LSTM(embedding_size, hidden_size, n_layers, dropout=dropout, batch_first=True)
    #self.dropout = nn.Dropout(0.3)
    self.fc = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = x.long()
    x = self.embedding(x)
    x = self.dropout1d(x)
    out, _ =  self.lstm(x)
    out = out[:, -1, :]
    #out = self.dropout(out)
    out = self.fc(out)
    out = self.sigmoid(out)

    return out

In [23]:
from torch.optim import Adam

vocab_size = num_words
output_size = 1
embedding_size = 256
hidden_size = 16
n_layers = 2
dropout = 0.5

lr = 0.001



# model initialization
model_checkpoint = LSTMModel(vocab_size, output_size, hidden_size, embedding_size, n_layers, dropout)
optim = Adam(model_checkpoint.parameters(), lr=lr)


checkpoint = torch.load('checkpoint.pth')
model_checkpoint.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model_checkpoint.eval()
print(model_checkpoint)



LSTMModel(
  (embedding): Embedding(10000, 256)
  (dropout1d): Dropout1d(p=0.5, inplace=False)
  (lstm): LSTM(256, 16, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [25]:
def review_pred(review, model):
  review = [review]
  for i in range(len(review)):
    review[i] = clean_review(review[i])

  review = tokenizer.texts_to_sequences(review)
  review = pad_sequences(review, maxlen=maxlen, dtype="int32")

  sentiment = model(torch.tensor(review).to(device)).to(device)

  pred = sentiment.cpu().detach().numpy()
  print(pred.item())

  if(pred < 0.5):
      print("negative")
  elif (pred >= 0.5):
      print("positive")

review = "this movie is great"

review_pred(review, model_checkpoint.to(device))

review1 = "this movie is terrible"

review_pred(review1, model_checkpoint.to(device))

review2 = "this movie is not terrible, it's great!"

review_pred(review2, model_checkpoint.to(device))

review3 = "this movie is not great, it's terrible!"

review_pred(review3, model_checkpoint.to(device))


0.7599336504936218
positive
0.7599336504936218
positive
0.7599336504936218
positive
0.7599336504936218
positive
